<center> <img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg" align="center" width="960" height="480"/> </center>

<font size=6> MICROESTRUCTURA Y SISTEMAS DE TRADING  </font> <br>

<font size = 5> Lab 4  </font><br>

<font size=5> IF Luis Enrique Sánchez Ruiz </font> <br>

<font size = 4> GitHub: <a href='https://github.com/luisen17/lab4.git'>Link</a></font>

# Laboratorio 4. Microestructura y Sistemas de Trading


In [3]:
import ccxt
import pandas as pd
import numpy as np

def descargaOHLCV(crypto):
    bequant = ccxt.bequant()
    datos = bequant.fetch_ohlcv(crypto, timeframe = "1m")
    
    for i in range(len(datos)):
        datos[i][0] = bequant.iso8601(datos[i][0])
        
    datosCrypto = pd.DataFrame(datos)
    datosCrypto.set_index([0], inplace = True)
    datosCrypto.columns = ["Open", "High", "Low", "Close", "Volume"]
    datosCrypto.index.name = "Timestamp"
    
    return datosCrypto

def descargaOrderBook(crypto, level):
    bequant = ccxt.bequant()
    datos = bequant.fetch_order_book(crypto, limit = level)
    datosCrypto = pd.DataFrame(index = [0], 
                           columns = ["Level", "Ask Volume", "Bid Volume", "Total Volume", "Mid Price", "VWAP"])

    datosCrypto["Level"] = level
    datosCrypto["Ask Volume"] = np.sum([ask[1] for ask in datos["asks"]])
    datosCrypto["Bid Volume"] = np.sum([bid[1] for bid in datos["bids"]])
    datosCrypto["Total Volume"] = np.sum([ask[1] for ask in datos["asks"]]) + np.sum([bid[1] for bid in datos["bids"]])
    datosCrypto["Mid Price"] = 0.5 * (-datos["bids"][0][0] + datos["asks"][0][0])
    datosCrypto["VWAP"] = (np.sum([ask[0] * ask[1] for ask in datos["asks"]]) + np.sum([bid[0] * bid[1] for bid in datos["bids"]])) / datosCrypto["Total Volume"]

    return datosCrypto

def effectiveSpreadRoll(crypto):
    crypto["Delta (Close)"] = crypto["Close"].diff()
    crypto["Spread Roll"] = 0

    for i in range(len(litecoin)):
        if i > 10:
            serie1 = crypto.iloc[i - 5 : i, 5].values
            serie2 = crypto.iloc[i - 10: i - 5, 5].values
            covarianza = np.cov(serie1, serie2)
        
            crypto.iloc[i, 6] = 2 * (abs(covarianza[1, 0])) ** 0.5
        
    return crypto

## Parte 1. CCXT

### LTC/USD: Litecoin

In [4]:
descargaOrderBook("LTCUSD", 10)

,Level,Ask Volume,Bid Volume,Total Volume,Mid Price,VWAP
0,10,69.358,89.658,159.016,0.0185,102.707998


### ADA/USD: Cardano

In [5]:
descargaOrderBook("ADAUSD", 10)

,Level,Ask Volume,Bid Volume,Total Volume,Mid Price,VWAP
0,10,7703.0,7378.0,15081.0,0.00005,0.833077


### BTC/USD: Bitcoin

In [6]:
descargaOrderBook("BTCUSD", 10)

,Level,Ask Volume,Bid Volume,Total Volume,Mid Price,VWAP
0,10,1.94066,1.7521,3.69276,2.03,39464.214061


## Parte 3. Modelado Microestructura: Effective Spread Roll

### LTC/USD: Litecoin

In [2]:
litecoin = descargaOHLCV("LTCUSD")
litecoin = effectiveSpreadRoll(litecoin)
litecoin = litecoin[["Close", "Spread Roll"]]
litecoin

,Close,Spread Roll
Timestamp,,
2014-02-07T18:21:00.000Z,15.000,0.000000
2014-02-08T20:03:00.000Z,20.000,0.000000
2014-02-11T12:58:00.000Z,18.438,0.000000
2014-02-11T13:31:00.000Z,18.252,0.000000
2014-02-11T14:15:00.000Z,18.260,0.000000
...,...,...
2014-02-12T10:27:00.000Z,16.736,0.126116
2014-02-12T11:11:00.000Z,16.947,0.022068
2014-02-12T11:21:00.000Z,16.881,0.111175


### ADA/USD: Cardano

In [3]:
cardano = descargaOHLCV("ADAUSD")
cardano = effectiveSpreadRoll(cardano)
cardano = cardano[["Close", "Spread Roll"]]
cardano

,Close,Spread Roll
Timestamp,,
2018-02-26T18:32:00.000Z,0.500000,0.000000
2018-02-26T18:34:00.000Z,0.500000,0.000000
2018-02-26T18:35:00.000Z,0.360000,0.000000
2018-02-26T18:37:00.000Z,0.500000,0.000000
2018-02-26T18:38:00.000Z,0.499999,0.000000
...,...,...
2018-02-26T23:29:00.000Z,0.340000,0.012539
2018-02-26T23:30:00.000Z,0.350000,0.012539
2018-02-26T23:31:00.000Z,0.345000,0.011991


### BTC/USD: Bitcoin

In [4]:
bitcoin = descargaOHLCV("BTCUSD")
bitcoin = effectiveSpreadRoll(bitcoin)
bitcoin = bitcoin[["Close", "Spread Roll"]]
bitcoin

,Close,Spread Roll
Timestamp,,
2013-12-27T08:23:00.000Z,777.00,0.000000
2013-12-27T08:24:00.000Z,778.00,0.000000
2013-12-27T08:25:00.000Z,777.00,0.000000
2013-12-27T08:26:00.000Z,778.00,0.000000
2013-12-27T08:34:00.000Z,779.00,0.000000
...,...,...
2013-12-27T19:58:00.000Z,700.10,47.056774
2013-12-27T19:59:00.000Z,731.98,55.802713
2013-12-27T20:00:00.000Z,735.83,57.792516
